<a href="https://colab.research.google.com/github/sungkwangsong/EasyOCR/blob/master/notebooks/hbn-find-articles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Repositories/Hibrainnet

/content/drive/MyDrive/Repositories/Hibrainnet


In [5]:
# 1. 필요한 라이브러리 설치
!pip install transformers
!pip install torch
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [25]:
#2. 환경변수 추가
HBN_DATA_DIR_PATH="/content/drive/MyDrive/Workspaces/Hibrainnet/hbn-scholar/data"
HBN_DATA_ARTICLES_DIR_PATH=f"{HBN_DATA_DIR_PATH}/articles"

In [34]:
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModel
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity


In [46]:
sample_papers = [
    {
        "title": "딥러닝 기반 한국어 자연어처리 기술 동향",
        "authors": ["김철수"],
        "co_authors": ["이영희", "박지민", "정동훈"],
        "abstract": "최근 딥러닝 기술의 발전으로 자연어처리 분야에서 큰 성과가 있었다. 특히 한국어 처리에 있어 형태소 분석, 개체명 인식, 감정 분석 등 다양한 태스크에서 성능이 크게 향상되었다. 본 논문에서는 한국어 자연어처리 분야의 최신 딥러닝 기술 동향을 살펴보고, 향후 발전 방향을 제시한다.",
        "journal": "한국정보과학회논문지",
        "year": 2022,
        "citations": 125,
        "keywords": ["자연어처리", "딥러닝", "한국어", "인공지능"]
    },
    {
        "title": "미세먼지가 호흡기 질환에 미치는 영향 연구",
        "authors": ["박성민"],
        "co_authors": ["김태호", "이준영"],
        "abstract": "대기오염물질 중 하나인 미세먼지(PM10)와 초미세먼지(PM2.5)가 인체의 호흡기 질환 발병률에 미치는 영향을 분석하였다. 서울시의 5년간의 대기질 측정 데이터와 호흡기 질환 진료 기록을 활용하여 상관관계를 도출하였으며, 특히 노약자층에서 유의미한 영향이 관찰되었다.",
        "journal": "대한환경의학회지",
        "year": 2023,
        "citations": 45,
        "keywords": ["미세먼지", "호흡기질환", "환경의학", "공중보건"]
    },
    {
        "title": "기계학습을 이용한 텍스트 마이닝 기반 감성분석",
        "authors": ["정보람"],
        "co_authors": ["송민준", "강현우"],
        "abstract": "소셜미디어 데이터에서 감성을 추출하기 위해 기계학습 알고리즘을 활용한 텍스트 마이닝 방법론을 제안한다. SVM, Random Forest, LSTM 등 다양한 알고리즘을 비교 실험하였으며, 특히 한국어 텍스트의 특성을 고려한 전처리 방법을 적용하여 성능을 개선하였다.",
        "journal": "한국빅데이터학회논문지",
        "year": 2023,
        "citations": 32,
        "keywords": ["텍스트마이닝", "감성분석", "기계학습", "소셜미디어"]
    },
    {
        "title": "친환경 건축물의 에너지 효율성 분석",
        "authors": ["이지원"],
        "co_authors": ["한동석", "임수진"],
        "abstract": "본 연구는 친환경 건축물의 에너지 효율성을 분석하고 개선 방안을 제시한다. 패시브 하우스 설계 요소가 건물의 에너지 소비량에 미치는 영향을 실증적으로 분석하였으며, 신재생 에너지 시스템의 통합 방안도 함께 검토하였다.",
        "journal": "대한건축학회논문집",
        "year": 2022,
        "citations": 67,
        "keywords": ["친환경건축", "에너지효율", "패시브하우스", "신재생에너지"]
    },
    {
        "title": "영유아의 스마트기기 사용이 언어발달에 미치는 영향",
        "authors": ["최유진"],
        "co_authors": ["박민서", "김도윤"],
        "abstract": "만 2-5세 영유아의 스마트기기 사용 실태를 조사하고 언어발달과의 관계를 분석하였다. 과도한 스마트기기 노출이 언어발달 지연과 관련이 있음을 확인하였으며, 적절한 사용 가이드라인과 대안적 언어활동을 제시한다.",
        "journal": "한국아동교육학회지",
        "year": 2023,
        "citations": 28,
        "keywords": ["영유아교육", "언어발달", "스마트기기", "아동발달"]
    },
    {
        "title": "인공지능 기반 학습자 맞춤형 교육 시스템 개발",
        "authors": ["장현수"],
        "co_authors": ["김민지", "이성호"],
        "abstract": "학습자의 성취도와 학습 패턴을 분석하여 개인화된 학습 콘텐츠를 제공하는 AI 기반 교육 시스템을 개발하였다. 머신러닝 알고리즘을 활용하여 학습자의 특성을 파악하고, 최적화된 학습 경로를 추천하는 방법을 제안한다.",
        "journal": "한국교육공학회논문지",
        "year": 2023,
        "citations": 52,
        "keywords": ["교육공학", "인공지능", "맞춤형교육", "이러닝"]
    },
    {
        "title": "한국 전통 발효식품의 항산화 활성 연구",
        "authors": ["김지현"],
        "co_authors": ["이상훈", "박소연"],
        "abstract": "김치, 된장, 고추장 등 한국 전통 발효식품의 항산화 활성을 분석하였다. 발효 과정에서 생성되는 다양한 생리활성 물질을 확인하고, 이들의 항산화 메커니즘을 규명하였다. 특히 유산균 발효에 의한 항산화 활성 증가 현상에 주목하였다.",
        "journal": "한국식품과학회지",
        "year": 2022,
        "citations": 89,
        "keywords": ["발효식품", "항산화", "전통식품", "식품과학"]
    }
]


# # JSON 파일로 저장
# with open(f'{HBN_DATA_PATH}/sample_papers.json', 'w', encoding='utf-8') as f:
    # json.dump(sample_papers, f, ensure_ascii=False, indent=2)





In [31]:
# 4. BERT 모델 및 토크나이저 로드
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [32]:
# 5. 논문 검색을 위한 클래스 정의
class PaperSearchEngine:
    def __init__(self, papers):
        self.papers = papers
        self.model = model
        self.paper_embeddings = None
        self._create_embeddings()

    def _create_embeddings(self):
        # 논문 제목과 초록을 결합하여 임베딩 생성
        texts = [f"{paper['title']} {paper['abstract']}" for paper in self.papers]
        self.paper_embeddings = self.model.encode(texts)

    def search(self, query, top_k=3):
        # 쿼리 임베딩 생성
        query_embedding = self.model.encode([query])

        # 코사인 유사도 계산
        similarities = cosine_similarity(query_embedding, self.paper_embeddings)[0]

        # 상위 k개 논문 찾기
        top_indices = np.argsort(similarities)[::-1][:top_k]

        results = []
        for idx in top_indices:
            results.append({
                'paper': self.papers[idx],
                'similarity_score': similarities[idx]
            })

        return results

In [44]:
# 6. 검색 엔진 초기화 및 테스트
def test_search_engine():
    # # JSON 파일 로드
    # with open('sample_papers.json', 'r', encoding='utf-8') as f:
        # papers = json.load(f)

    # 검색 엔진 초기화
    search_engine = PaperSearchEngine(sample_papers)

    # 테스트 쿼리 실행
    test_query="한국어 자연어처리 감성분석"
    results = search_engine.search(test_query)

    # 결과 출력
    print(f"Query: {test_query}\n")
    for i, result in enumerate(results, 1):
        paper = result['paper']
        score = result['similarity_score']
        print(f"Result {i}:")
        print(f"Title: {paper['title']}")
        print(f"Authors: {', '.join(paper['authors'] + paper['co_authors'])}")
        print(f"Year: {paper['year']}")
        print(f"Similarity Score: {score:.4f}")
        print()


In [47]:
# 테스트 실행
test_search_engine()

Query: 한국어 자연어처리 감성분석

Result 1:
Title: 영유아의 스마트기기 사용이 언어발달에 미치는 영향
Authors: 최유진, 박민서, 김도윤
Year: 2023
Similarity Score: 0.5697

Result 2:
Title: 딥러닝 기반 한국어 자연어처리 기술 동향
Authors: 김철수, 이영희, 박지민, 정동훈
Year: 2022
Similarity Score: 0.5334

Result 3:
Title: 친환경 건축물의 에너지 효율성 분석
Authors: 이지원, 한동석, 임수진
Year: 2022
Similarity Score: 0.5293



In [37]:

# 7. 대화형 검색 인터페이스
def interactive_search():
    # # 검색 엔진 초기화
    # with open('sample_papers.json', 'r', encoding='utf-8') as f:
        # papers = json.load(f)
    search_engine = PaperSearchEngine(sample_papers)

    while True:
        query = input("\nEnter your search query (or 'quit' to exit): ")
        if query.lower() == 'quit':
            break

        results = search_engine.search(query)

        print("\nSearch Results:")
        for i, result in enumerate(results, 1):
            paper = result['paper']
            score = result['similarity_score']
            print(f"\nResult {i}:")
            print(f"Title: {paper['title']}")
            print(f"Authors: {', '.join(paper['authors'] + paper['co_authors'])}")
            print(f"Journal: {paper['journal']}")
            print(f"Year: {paper['year']}")
            print(f"Citations: {paper['citations']}")
            print(f"Similarity Score: {score:.4f}")
            print(f"Abstract: {paper['abstract'][:200]}...")

# 메인 실행 코드
if __name__ == "__main__":
    print("Testing search engine with sample query...")
    test_search_engine()

    print("\nStarting interactive search...")
    interactive_search()

Testing search engine with sample query...
Query: transformer architecture for natural language processing

Result 1:
Title: Attention Is All You Need
Authors: Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit
Year: 2017
Similarity Score: 0.4659

Result 2:
Title: BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding
Authors: Jacob Devlin, Ming-Wei Chang, Kenton Lee, Kristina Toutanova
Year: 2019
Similarity Score: 0.4295

Result 3:
Title: GPT-3: Language Models are Few-Shot Learners
Authors: Tom B. Brown, Benjamin Mann, Nick Ryder, Melanie Subbiah
Year: 2020
Similarity Score: 0.2973


Starting interactive search...


KeyboardInterrupt: Interrupted by user